In [6]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")


import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd


device='cuda'

size=1000
batch=4
epochs=1

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
train_dataset=ds["validation"].shuffle(seed=40).select(range(size))

train_data=[]
for i in range(size):
    # tokenized = tokenizer(train_dataset[i]['context']+train_dataset[i]['question'] + train_dataset[i]['answers']['text'][0], padding="max_length", max_length=512, truncation=True, return_tensors="pt")
    # cqlen = len(tokenizer(train_dataset[i]['context']+train_dataset[i]['question'])['input_ids'])
    tokenized = tokenizer('C: '+train_dataset[i]['context']+' Q: '+train_dataset[i]['question'] +' A: '+ train_dataset[i]['answers']['text'][0], padding="max_length", max_length=512, truncation=True, return_tensors="pt")
    cqlen = len(tokenizer('C: '+train_dataset[i]['context']+' Q: '+train_dataset[i]['question'])['input_ids'])
    length=torch.sum(tokenized['attention_mask'][0])
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]

    labels=[-100]*512
    for i in range(length-cqlen):
        labels[cqlen+i-1]=input_ids[cqlen+i]
    
    labels = [-100 if x in (362, 25) else x for x in labels]
    
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

In [7]:
my_label=[]
for i in train_data:
    my_label.append(i['labels'])

In [8]:
labelnums=[]
for data in my_label:
    labelnum=[]
    for num in data:
        if num != -100:
            labelnum.append(num)
    labelnums.append(labelnum)
labelnums

[[93134],
 [59239, 331, 2191],
 [1749, 315, 63907, 6082],
 [220, 6393, 23],
 [98492, 1413, 8798, 20858],
 [279, 2447, 31384],
 [22578, 21787],
 [92162, 613],
 [42249, 52198],
 [6063],
 [220, 10288, 19],
 [6328, 58369],
 [23965, 6989],
 [220, 9741, 16],
 [220, 9263, 22],
 [279, 3279, 858, 315, 452, 15844],
 [220, 17, 1774, 264, 749],
 [66774, 55488],
 [11542, 315, 22260],
 [55580, 5435],
 [264,
  15664,
  449,
  9580,
  220,
  18,
  26545,
  311,
  7195,
  264,
  6883,
  502,
  29054,
  4009],
 [15395, 13255, 81149],
 [9343],
 [1469, 55094, 9289, 276, 14228, 19338],
 [220, 7285, 22],
 [8219],
 [7100, 25768],
 [20742, 276, 20742, 276],
 [279, 1455, 47999, 11, 19428],
 [47809, 50635],
 [453, 6885],
 [12656, 469, 13, 22021],
 [4848],
 [36250, 220, 23],
 [88940],
 [10690, 8669],
 [18880, 38373, 26545],
 [11756],
 [4606],
 [14763],
 [88264],
 [99690, 582, 2676, 35291],
 [994, 814, 7417, 8396, 439, 264, 4459],
 [60759, 386, 74187],
 [220, 6393, 18],
 [7487, 304, 279, 8009, 5961],
 [578,
  319

In [9]:
for data in labelnums:
    print(tokenizer.decode(data))

 anarchists
 catechism
 method of locomotion
 1948
 ablative heat shield
 the project coordinator
 Steam engines
 Fußach
 tenggis
 German
 1474
 ONdigital
 complexity classes
 1851
 1317
 the Edict of Nantes
 245 a.m
 Aston Webb
 freedom of ceremony
 computational problems
 a partnership with Level 3 Communications to launch a brand new nationwide network
 kilogram-force
 Session
 apicomplexan-related diseases
 1937
 Sun
 economic separation
 Duran Duran
 the most rigorous, intense
 Owen Daniels
 alga
 William E. Simon
 six
 Apollo 8
 unmanned
 Brownlee
 Capital Cities Communications
 coal
 Europe
 destroyed
 Serbian
 oxyacetylene
 when they improve society as a whole
 Mohamed Morsi
 1943
 workers in the poor countries
 The plan that the delegates agreed to was never ratified by the colonial legislatures nor approved of by the crown
 Midsummer’s Night
 computation time
 Catholic
 Peyton Manning
 after the Franco-German War
 to legalize importation of medications from Canada and other c